### Transformers

- documentacion y fuentes:
- - https://github.com/codificandobits/Traductor_con_redes_Transformer
- - https://github.com/CyberZHG/keras-transformer
- -  https://arxiv.org/abs/1706.03762 ( google )

### Diferencias entre secuenciales y transformers

**Secuenciales**:
 - memoria a corto plazo (incluso si son LSTM).
 - Procesado palabra por palabra

 - Memoria a largo plazo usando un mecanismo llamado **atención**
 - procesamiento en paralelo.
  

### Proceso:

<img src = './images/diagrama_transformers.png'>

# Encoding

- Embedding Entrada: Convierte el texto en una serie de vectores, en tokens (representación numerica).


- Codificacion de posicion: La secuencia se procesa en paralelo, es necesario indicar el orden en que se encuentran las palabras en el texto. Este codificador de posiciones genera vectores que se sumaran a los tokens y que indican la posicion relativa de cada token dentro de la secuencia.

(pueden existir varios :))
- **Bloque de codificacion**.
  - - **atencional**: se encarga de analizar la totalidad de la secuencia y buscar relaciones a diferentes niveles dentro de la secuencia.
Los tokens se llevan a 3 redes neuronales entrenadas para encontrar 3 vectores. **Queries**, **keys** y **values**. (3 representaciones alternativas de los tokes originales).

Luego cada queries se compara con los keys, con esto se obtiene un puntaje para el grado de asociacion de pares de palabras 
(por ejemplo con similitud de coseno.). Con estos puntajes se pondera cada uno de los vectores **values**
<center>
<img src = 'images/atencional.png'>
</center>

Es necesario escalar los puntajes, escalandolos dividiendolo entre el tamaño de cada vector y llevandolo a una función softmax.

<center>
<img src = './images/diagrama_transformers_2.png'>
</center>

Luego se multiplica esta matrix con la matriz de values. Estos serán (tantos como palabras hay) nuevos tokes con la codificación de la información de contexto más relevante para cada palabra de la secuencia.

**nota:** Un bloque atencional no es suficiente. Ya que por ejemplo "I Love Euclidean Geometry" puede tener varias asociaciones en un bloque atencional.  [I Love] [Euclidean Geometry] o [I] [Love] [Euclidean Geometry]. Es decir además de asociaciones entre palabras hay asociaciones entre frases.

Si se usan multiples bloques atencionales, es posible encontrar asociaciones de palabras y frases en diferente niveles. Luego si se tienen $k$ bloques atencionales, estos se combinan con una red neuronal en un único vector por cada token

<center>
<img src = './images/diagrama_transformers_3.png'>
</center>



  - -  **bloque residual:** En este bloque se lleva tanto la entrada como la salida del bloque atencional. Luego suma y normaliza estos datos para tener una escala apropiada para el siguiente bloque.

  <center>
<img src = './images/diagrama_transformers_4.png'>
</center>


  - - **red neuronal + bloque residual:**
  La red neuronal procesa en paralelo todos los vectores de la secuencia. tomanda la info atencional de las capas anteriores y consolidandolas en una única representación. La entrada y salida de esta red neuronal son enviadas a un bloque residual que tiene las mismas caracteristicas del bloque anterior

  






# Decoding

<center>
<img src = 'images/decodificador_1.png'>
</center>


- **Embedding Salida** y **Codificación posicional**. (analogo a los anteriores anterior)

- **Bloque Atencional con enmascaramiento** Codifica la relación entre diferentes elementos de salida, usando los **Queries** , **Keys** y **Values**, la gran diferencia es que cada palabra se genera de manera secuencial, el decodificador solo debe prestar atención debe prestar atención **unicamente** alas palabras generas anteriormente y no a las futuras

<center>
<img src = 'images/decodificador_2.png'>
</center>

- **Bloque atencional (decodificador)** A diferencia del bloque atencional anterior (codificación) este bloque centra su atención en la secuencia original como la de salida. Toma la salida del codificador a las redes **Queries** y **Keys**. Mientras que la red **values** tiene como entrada el dato proveniente del bloque residual anterior.

Asi el codificar le comunica al decodificador a que elementos debe prestar mas atencios al momemnto de generar la secuencia de salida (analogamente se toman muchos bloques atencionales para codificar asociaciones). Luego de pasar por los decodificadores
genera un vector con cantidades numericas.

**Capa Lineal** Toma el resultado del decodificador y lo transforma en un vector más grande. Por ejemplo si el traductor aprende 10.000 palabras, entonces el vector de salida de la capa lineal tendra 10.000 elementos

**Softmax** Esta capa convierte cada elemento de este vector en sus respectivas probabilidades.

## Ejemplo traductor (ingles a español)
 

In [123]:
import numpy as np
from keras_transformer import get_model, decode
from pickle import load

np.random.seed(0)

### Carga de data

In [124]:
filename = 'data/english-spanish.pkl'
df = load(open(filename , 'rb'))

In [125]:
df = df[:130,:]

In [126]:
# print( f'palabra en ingles: {df[1221,0]}')
# print( f'palabra en español: {df[1221,1]}')

### Embedding

In [127]:
# Creacion de "tokens"
# Transforma cada palabra en una lista de palabras, esto por cada palabra.

# tokens en ingles
source_tokens = []
for sentencia in df[:, 0]:
    source_tokens.append(sentencia.split(' '))

# Tokens en español
target_tokens = []
for sentencia in df[:, 1]:
    target_tokens.append(sentencia.split(' '))

print(f'palabra en ingles: {df[120,0]}')
print(f'palabra en ingles tokenizada: {source_tokens[120]}\n')
print(f'palabra en español: {df[120,1]}')
print(f'palabra en español tokenizada: {target_tokens[120]}')

palabra en ingles: be good
palabra en ingles tokenizada: ['be', 'good']

palabra en español: sean buenos
palabra en español tokenizada: ['sean', 'buenos']


In [139]:
def build_token_dict(token_list):
    """Resumen

    Args:
        token_list: Una lista que contiene lista de frases tokenizadas.

    Returns:
        dict: retorna un diccionario, donde el 0 es padding, 1 es start 2 end, y el resto de palabras (todas diferentes) tiene como como key un único numero (el tamaño del diccionario antes de ser agregada).
    """
    
    token_dict = {
        '<PAD>': 0,
        '<START>': 1,
        '<END>': 2
    }
    for tokens in token_list:
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = len(token_dict)
    return token_dict
        

In [140]:
source_tokens_dict = build_token_dict(source_tokens)
target_tokens_dict = build_token_dict(target_tokens)
target_tokens_dict

{'<PAD>': 0,
 '<START>': 1,
 '<END>': 2,
 've': 3,
 'vete': 4,
 'vaya': 5,
 'vayase': 6,
 'hola': 7,
 'corre': 8,
 'corran': 9,
 'corra': 10,
 'corred': 11,
 'quien': 12,
 'orale': 13,
 'fuego': 14,
 'incendio': 15,
 'disparad': 16,
 'ayuda': 17,
 'socorro': 18,
 'auxilio': 19,
 'salta': 20,
 'salte': 21,
 'parad': 22,
 'para': 23,
 'pare': 24,
 'espera': 25,
 'esperen': 26,
 'continua': 27,
 'continue': 28,
 'date': 29,
 'prisa': 30,
 'daos': 31,
 'dese': 32,
 'me': 33,
 'oculte': 34,
 'escondi': 35,
 'ocultaba': 36,
 'escondia': 37,
 'corri': 38,
 'corria': 39,
 'lo': 40,
 'intento': 41,
 'he': 42,
 'ganado': 43,
 'oh': 44,
 'no': 45,
 'tomatelo': 46,
 'con': 47,
 'soda': 48,
 'disparen': 49,
 'dispara': 50,
 'dispare': 51,
 'sonrie': 52,
 'al': 53,
 'ataque': 54,
 'atacad': 55,
 'ataquen': 56,
 'ataca': 57,
 'levanta': 58,
 'ahora': 59,
 'mismo': 60,
 'id': 61,
 'vayan': 62,
 'ya': 63,
 'tengo': 64,
 'pillas': 65,
 'entendiste': 66,
 'el': 67,
 'corrio': 68,
 'metete': 69,
 'adentro

In [141]:
## Diccionario con llave el label de la palabra y key la palabra
target_tokens_dict_inv = {palabra: label for label,
                          palabra in target_tokens_dict.items()}

In [142]:
source_tokens_dict

{'<PAD>': 0,
 '<START>': 1,
 '<END>': 2,
 'go': 3,
 'hi': 4,
 'run': 5,
 'who': 6,
 'wow': 7,
 'fire': 8,
 'help': 9,
 'jump': 10,
 'stop': 11,
 'wait': 12,
 'on': 13,
 'hello': 14,
 'hurry': 15,
 'i': 16,
 'hid': 17,
 'ran': 18,
 'try': 19,
 'won': 20,
 'oh': 21,
 'no': 22,
 'relax': 23,
 'shoot': 24,
 'smile': 25,
 'attack': 26,
 'get': 27,
 'up': 28,
 'now': 29,
 'got': 30,
 'it': 31,
 'he': 32,
 'hop': 33,
 'in': 34,
 'hug': 35,
 'me': 36,
 'care': 37,
 'fell': 38,
 'fled': 39,
 'know': 40,
 'left': 41,
 'lied': 42,
 'lost': 43,
 'quit': 44,
 'sang': 45,
 'wept': 46,
 'work': 47,
 'im': 48,
 'listen': 49,
 'way': 50,
 'really': 51,
 'thanks': 52,
 'we': 53,
 'why': 54,
 'ask': 55,
 'tom': 56,
 'awesome': 57,
 'be': 58,
 'calm': 59,
 'cool': 60,
 'fair': 61,
 'good': 62,
 'kind': 63,
 'nice': 64,
 'beat': 65,
 'call': 66,
 'us': 67,
 'come': 68}

In [143]:
# Agregar start, end y pad a cada frase del entrenamiento

encoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]

# salida del transformer.
output_tokens = [tokens + ['<END>'] for tokens in target_tokens]

# Tamaño maximo de una palabra de entrada y de salida
source_max_len = max(map(len, encoder_tokens))
target_max_len = max(map(len, decoder_tokens))

## Tenemos que entregar una lista con la misma dimensión. Dado que hay textos con diferentes tamaños 
encoder_tokens = [tokens + ['<PAD>'] *
                  (source_max_len - len(tokens)) for tokens in encoder_tokens]

decoder_tokens = [tokens + ['<PAD>'] *
                  (target_max_len - len(tokens)) for tokens in decoder_tokens]


## La salida final será en español por lo que se llenan espacios con
output_tokens = [tokens + ['<PAD>'] *
                  (source_max_len - len(tokens)) for tokens in output_tokens]



### Codificación posicional

In [144]:
# transformar cada palabra en su respectiva etiqueta.
encoder_input = [list(map(lambda x: source_tokens_dict[x], tokens))
                 for tokens in encoder_tokens]

decoder_input = [list(map(lambda x: target_tokens_dict[x], tokens))
                 for tokens in decoder_tokens]

## TODO revisar mas adelante minuto 29:41

output_decoded = [list(map(lambda x: [target_tokens_dict[x]], tokens))  
                  for tokens in output_tokens]

144

### Transformers

- **token_num** : Tamaño máximo que puede llegar a tener cada frase.
- **embed_dim** : Dimensión del embedding de entrada
- **encoder_num**: Número de codificadores que tiene la entrada.
- **decoder_num**: Número de decodificadores que tiene la salida
- **head_num**: Número de bloques atencionales (para encontrar relaciones en diferentes niveles)
- **hidden_dim**: Número de neuronas en la capa oculta (La capa verde en la teoria)
- **dropout_rate**: Porcentajes de neuronas desactivadas aleatoriamente durante el proceso
- **use_same_embed** (Boolean) donde "False" indica que la forma de representar la frase en ingles va a ser diferente a la forma en que va a encontrar las frases en español.
 


In [145]:
source_tokens_dict

{'<PAD>': 0,
 '<START>': 1,
 '<END>': 2,
 'go': 3,
 'hi': 4,
 'run': 5,
 'who': 6,
 'wow': 7,
 'fire': 8,
 'help': 9,
 'jump': 10,
 'stop': 11,
 'wait': 12,
 'on': 13,
 'hello': 14,
 'hurry': 15,
 'i': 16,
 'hid': 17,
 'ran': 18,
 'try': 19,
 'won': 20,
 'oh': 21,
 'no': 22,
 'relax': 23,
 'shoot': 24,
 'smile': 25,
 'attack': 26,
 'get': 27,
 'up': 28,
 'now': 29,
 'got': 30,
 'it': 31,
 'he': 32,
 'hop': 33,
 'in': 34,
 'hug': 35,
 'me': 36,
 'care': 37,
 'fell': 38,
 'fled': 39,
 'know': 40,
 'left': 41,
 'lied': 42,
 'lost': 43,
 'quit': 44,
 'sang': 45,
 'wept': 46,
 'work': 47,
 'im': 48,
 'listen': 49,
 'way': 50,
 'really': 51,
 'thanks': 52,
 'we': 53,
 'why': 54,
 'ask': 55,
 'tom': 56,
 'awesome': 57,
 'be': 58,
 'calm': 59,
 'cool': 60,
 'fair': 61,
 'good': 62,
 'kind': 63,
 'nice': 64,
 'beat': 65,
 'call': 66,
 'us': 67,
 'come': 68}

In [174]:
from keras.layers import Embedding


model = get_model(
    token_num=max(len(source_tokens_dict), len(target_tokens_dict)),
    embed_dim=32,
    encoder_num=3,
    decoder_num=2,
    head_num=4,
    hidden_dim=128,
    dropout_rate=0.05,
    use_same_embed=False,
)

# model.compile('adam','sparse_categorical_crossentropy')

ValueError: Unrecognized keyword arguments passed to EmbeddingRet: {'weights': None}

In [ ]:
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

In [171]:
## Entrenamiento
x = [np.array(encoder_input), np.array(decoder_input)]
y = [np.array(output_decoded)]

# No tengo suficiente memoria :( 
# model.fit(x,y, epochs=15, batch_size= 32)

filename_model = 'models/translator.h5'
model.load_weights(filename_model)

NameError: name 'model' is not defined

In [ ]:
def translate(sentencia):
    sentencia_tokens = [tokens + ['<END>', '<PAD>']
                        for tokens in [sentencia.split(' ')]]
    tr_input = [list(map(lambda x: source_tokens_dict[x], tokens))
                for tokens in sentencia_tokens]

    # decodificacion
    decoded = decode(
        model,
        tr_input,
        start_token=target_tokens_dict['<START>'],
        end_token=target_tokens_dict['<END>'],
        pad_token=target_tokens_dict['<PAD>']
    )
    print(f"Frase original {sentencia}")
    
    print(f"Frase traducida {' '.join(map(lambda x :target_tokens_dict_inv[x], decoded[1:-1]))}")

- **tr_input**: Es la entrada del transformer, agregando < END>  y < PAD> (START no es necesario)

- **decoded[1:-1]** me va a retornar los label de cada frase traducida, luego obtenemos la palabra correspondiente en target_tokens_dict_inv. 

- **start_token , end_token, pad_token**: deben ser los label que tomamos para indicar donde empieza una frase,donde termina y donde dejamos espacios respectivamente.


In [ ]:
translate("The red car")

ValueError: Unrecognized keyword arguments passed to EmbeddingRet: {'weights': [array([[6.00212868e-01, 9.63197295e-01, 1.47801334e-01, 2.56916644e-01,
        8.73556827e-01, 4.91892232e-01, 8.98961092e-01, 1.85517898e-01,
        5.32668587e-01, 3.26269633e-01, 3.16542560e-01, 4.46876964e-01,
        4.33077449e-01, 3.57346880e-01, 9.14970770e-01, 7.31744185e-01,
        7.27546991e-01, 2.89913450e-01, 5.77709424e-01, 7.79179433e-01,
        7.95590369e-01, 3.44530461e-01, 7.70872757e-01, 7.35893897e-01,
        1.41506486e-01, 8.65945469e-01, 4.41321470e-01, 4.86410449e-01,
        4.48369179e-01, 5.67846001e-01],
       [6.21169247e-01, 4.98179566e-01, 8.66788543e-01, 6.27734756e-01,
        4.01427949e-01, 4.16691757e-01, 8.10838615e-01, 3.48191943e-01,
        2.11454796e-01, 5.93831880e-02, 8.76026848e-01, 9.18546451e-01,
        1.20120182e-01, 3.34473741e-01, 1.75372070e-01, 1.15898469e-01,
        8.99866743e-01, 5.68772591e-02, 9.80485663e-01, 9.64508607e-02,
        8.63470